In [1]:
import pandas as pd 
import numpy as np

class main():
    def __init__(self,f1):
        self.df=pd.read_csv(f1)
        self.column_list=self.df.columns.values
        self.null_columns=[i for i in self.column_list if self.df[i].isnull().any()==True]
        self.sizedf=np.size(self.df,axis=0)

        self.diff=list(set(self.column_list) - set(self.null_columns))

        self.int_list=[j for j in self.diff if self.df[j].dtype in (int,np.short,np.int64,np.int16,np.int32)]
        self.float_list=[k for k in self.diff if self.df[k].dtype in (float,np.double,np.float64,np.longdouble,np.longfloat,np.float32)]
        self.string_list=[l for l in self.diff if self.df[l].dtype in (str,object)]

        self.int_nulllist=[j for j in self.null_columns if self.df[j].dtype in (int,np.short,np.int64,np.int16,np.int32)] 
        self.float_nulllist=[k for k in self.null_columns if self.df[k].dtype in (float,np.double,np.float64,np.longdouble,np.longfloat,np.float32)]
        self.string_nulllist=[l for l in self.null_columns if self.df[l].dtype in (str,object)]
    
    def retval(self):
        return self.df

class categorical():
    def check(self,x):
        tlist=[]
        for i in x.columns:
            for j in x[i]:
                if j.isnumeric():
                    tlist.append(i)
        return tlist

    def cat_null(self,x,dt):
        x=pd.DataFrame(x)
        x.dropna(axis=0,inplace=True)
        if dt==str:    
            cval=self.check(x)
            if len(cval)>0:
                try:
                    y=pd.DataFrame(x[cval],dtype=int)
                    x.drop(cval,inplace=True)
                    x.merge(y,left_index=True,right_index=True)
                except:
                    pass
                pass
        truevalue_list=[]
        for i in x.columns:
            if (len(x[i].unique())>0 and len(x[i].unique())<=10):
                truevalue_list.append(i)
        return (truevalue_list)

    def cat_fill(self,x):
        # for all null values.
        x=pd.DataFrame(x)
        for c in x.columns:
            valcnt=x[c].value_counts(dropna=True)
            valcntidx=valcnt.index
            #filling it with max or min depends on size and bias based off of how the categories are distributed, 
            #here by default we try to support the max class
            maxval=valcnt.max()


            for i in valcntidx:
                if valcnt[i]==maxval:
                    x[c]=x[c].fillna(i)
                    break
        return x

class noncategorical():
    def ncat_ifill(self,x,l):
        x=pd.DataFrame(x)
        x1=x.dropna(axis=0)
        for i in x.columns:
            meanvalue=x1[i].mean()
            x[i].fillna(round(meanvalue))


    def ncat_sfill(self,x):
        x=x.fillna("-")
        #drop non cat strings
        pass

    def ncat_ffill(self,x):
        x=pd.DataFrame(x)
        x1=x.dropna(axis=0)
        for i in x.columns:
            meanvalue=x1[i].mean()
            x[i]=x[i].fillna(meanvalue)
        return x

class null(categorical,noncategorical,main):
    def __init__(self,f):
        super().__init__(f)

        self.intfill()
        self.stringfill()
        self.floatfill()
        
        self.mainint=[j for j in self.df.columns if self.df[j].dtype in (int,np.short,np.int64,np.int16,np.int32)]
        self.mainfloat=[k for k in self.df.columns if self.df[k].dtype in (float,np.double,np.float64,np.longdouble,np.longfloat,np.float32)]
        self.mainstring=[l for l in self.df.columns if self.df[l].dtype in (str,object)]
        
        for i in self.mainint:
            self.df[i].apply(lambda x: round(x))

        for j in self.mainfloat:
            self.df[j].apply(lambda x: round(x,2))

        for k in self.mainstring:
            self.df[k].apply(lambda x: x.strip())
            self.df[k].apply(lambda x: x.capitalize())

    def intfill(self):
        intval=self.cat_null(self.df[self.int_nulllist],int)
        difference_int=list(set(self.int_nulllist)-set(intval)) #use difference in both lists to call non categorical val

        if len(intval)==0:
#             print("no val")
            pass
        else:
            self.df[intval]=self.cat_fill(self.df[intval])
#             print(df[intval].isnull().any())

        if len(difference_int)==0:
#             print("no val")
            pass
        else:
            self.df[difference_int]=self.ncat_ifill(self.df[difference_int])
#             print(df[difference_int].isnull().any())

    def stringfill(self):
        stringval=self.cat_null(self.df[self.string_nulllist],str)
        difference_string=list(set(self.string_nulllist)-set(stringval))

        if len(stringval)==0:
#             print("no val")
            pass
        else:
            self.df[stringval]=self.cat_fill(self.df[stringval])
#             print(df[stringval].isnull().any())

        if len(difference_string)==0:
#             print("no val")
            pass
        else:
            self.df[difference_string]=self.ncat_sfill(self.df[difference_string])
#             print(df[difference_string].isnull().any())

    def floatfill(self):
        floatval=self.cat_null(self.df[self.float_nulllist],float)
        difference_float=list(set(self.float_nulllist)-set(floatval))

        if len(floatval)==0:
#             print("no val")
            pass
        else:
            self.df[floatval]=self.cat_fill(self.df[floatval])
#             print(df[floatval].isnull().any())

        if len(difference_float)==0:
#             print("no val")
            pass
        else:
            self.df[difference_float]=self.ncat_ffill(self.df[difference_float])
#             print(df[difference_float].isnull().any())



In [2]:
def gmain(f):
    obj1=null(f.name)
    r=obj1.retval()
    r.to_csv("out.csv")
    return "out.csv"

import gradio as gr
g=gr.Interface(gmain,inputs=["file"],outputs="file")
g.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x2124b10d370>, 'http://127.0.0.1:7860/', None)